In [1]:
from Preprocessing.MECO_data_split import concat_MECO_langs, split_into_time_series, lang_list

In [2]:
data = concat_MECO_langs(['ru'])
X, X_test, y, y_test, demo, demo_test = split_into_time_series(data)

In [3]:
X_flat = X.reshape((X.shape[0], 540))
X_test_flat = X_test.reshape((X_test.shape[0], 540))

In [4]:
%load_ext tensorboard
import keras
from keras import layers

import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [5]:
input_img = keras.Input(shape=540)
encoded = layers.Dense(128, activation='relu')(input_img)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(32, activation='relu')(encoded)

decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(540, activation='sigmoid')(decoded)

In [6]:
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(X_flat, X_flat,
               epochs=10,
               batch_size=32,
               shuffle=True,
               validation_data=(X_test_flat, X_test_flat))

Epoch 1/100


2023-05-14 13:06:50.802969: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2/2 [==============================] - 1s 88ms/step - loss: -812.3001 - val_loss: -15171.4102
Epoch 2/100
2/2 [==============================] - 0s 12ms/step - loss: -20427.8945 - val_loss: -56627.7148
Epoch 3/100
2/2 [==============================] - 0s 13ms/step - loss: -67595.2578 - val_loss: -140455.0312
Epoch 4/100
2/2 [==============================] - 0s 14ms/step - loss: -161160.0781 - val_loss: -294015.2188
Epoch 5/100
2/2 [==============================] - 0s 14ms/step - loss: -330190.5000 - val_loss: -556476.5000
Epoch 6/100
2/2 [==============================] - 0s 14ms/step - loss: -614662.5000 - val_loss: -982887.9375
Epoch 7/100
2/2 [==============================] - 0s 16ms/step - loss: -1074720.2500 - val_loss: -1654812.5000
Epoch 8/100
2/2 [==============================] - 0s 14ms/step - loss: -1798498.0000 - val_loss: -2690132.7500
Epoch 9/100
2/2 [==============================] - 0s 14ms/step - loss: -2912696.2500 - val_loss: -4250186.5000
Epoch 10/100
2/2 [=====